In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import wandb
import time
import os
from tqdm import tqdm
import pandas as pd
import transformers
import torch
import numpy as np
import random

from tqdm import tqdm
from sklearn.utils import shuffle
from transformers import (DebertaForSequenceClassification,
                          Trainer,
                          TrainingArguments,
                          DebertaTokenizerFast,
                          PegasusTokenizer)
from pattern.en import lexeme
from sentence_transformers import SentenceTransformer, util
from trl.simplification_reward import SimplificationReward
from trl.pegasus import PegasusWithValueHeadModel, respond_to_batch_pegasus
from trl.ppo_v2 import PegasusPPOTrainer
from trl.core import split_batch_encoding


torch_device = 'cuda:2' if torch.cuda.is_available() else 'cpu'

In [3]:
config = {
    "lm_name": 'tuner007/pegasus_paraphrase',
    "ref_lm_name": 'tuner007/pegasus_paraphrase',
    "tk_name": "tuner007/pegasus_paraphrase",
    "steps": 1500,
    "batch_size": 8,
    "forward_batch_size": 4,
    "ppo_epochs": 4,   
    "txt_in_len": 100,
    "txt_out_len": 50,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
    "device_ppo": 'cuda:1',
    "device_ref": 'cuda:0',
    "reward_coef": 0.05,
    "alpha": 1.0,
    "beta": 0.5,
    "optimizer": "SGD"
}

### Loading Data

In [4]:
asset_valid_path_ref = '/home/m25dehgh/simplification/datasets/asset/dataset/ref-valid/asset.valid.simp.1'
asset_valid_path_orig = '/home/m25dehgh/simplification/datasets/asset/dataset/asset.valid.orig'
wikilarge_train_path_ref = '/home/m25dehgh/simplification/datasets/wikilarge/data-simplification/wikilarge/wiki.full.aner.ori.train.dst'
wikilarge_train_path_orig = '/home/m25dehgh/simplification/datasets/wikilarge/data-simplification/wikilarge/wiki.full.aner.ori.train.src'
newsela_auto_train_path_orig = "/home/m25dehgh/simplification/datasets/newsela-auto/newsela-auto/newsela-auto/ACL2020/train.src"
newsela_auto_train_path_ref = "/home/m25dehgh/simplification/datasets/newsela-auto/newsela-auto/newsela-auto/ACL2020/train.dst"


In [5]:
# orig_wiki_sents = open(wikilarge_train_path_orig).read().split('\n')
# ref_wiki_sents = open(wikilarge_train_path_ref).read().split('\n')

In [6]:
x_newsela = open(newsela_auto_train_path_orig).read().split('\n')
y_newsela = open(newsela_auto_train_path_ref).read().split('\n')
x_newsela, y_newsela = shuffle(x_newsela, y_newsela)

smaple_size = 50000
i = random.randrange(len(x_newsela) - smaple_size)
orig_newsela_auto_sents = x_newsela[i:i+smaple_size]
ref_newsela_auto_sents = y_newsela[i:i+smaple_size]

In [7]:
orig_newsela_auto_sents[100], ref_newsela_auto_sents[100]

('Public health groups such as the American Academy of Pediatrics embraced the guidelines , but food manufacturers , media companies and trade associations mounted a furious pushback .',
 'But food makers and television companies fought them .')

### Loading the Reward Model

In [8]:
# root_comp_simp = "/home/m25dehgh/simplification/complex-classifier"
# model_comp_simp = "newsela-auto-high-quality"

# # path to saved checkpoint of deberata model trained on simple and complex corpora
# path_comp_simp = root_comp_simp + '/results' + '/' + model_comp_simp + "/whole-high-quality/checkpoint-44361/"
# comp_simp_class_model = DebertaForSequenceClassification.from_pretrained(path_comp_simp).to(torch_device)
# comp_simp_class_model.eval()
# tokenizer_deberta = DebertaTokenizerFast.from_pretrained('microsoft/deberta-base')
# semantic_model = SentenceTransformer('all-mpnet-base-v2', device=torch_device)

In [9]:

# input_a = ['below are some useful links to facilitate your involvement.',
#           'below are some useful links to facilitate your involvement.',
#           'A massive glacier had crashed down the mountain.',]
# input_b = ['below are some useful links to make it easier for you to get involved.',
#           'below are some useful links to facilitate your involvement.',
#           'A big glacier had crashed down the mountain.']

# model_reward = SimplificationReward(comp_simp_class_model, tokenizer_deberta, semantic_model, **config)
# model_reward.cal_reward(input_a, input_b)

### Loading Pegasus Paraphrasing Model with Value head

In [10]:
pegasus_model = PegasusWithValueHeadModel.from_pretrained(config['lm_name']).to(config['device_ppo'])
# pegasus_model.train()

pegasus_model_ref = PegasusWithValueHeadModel.from_pretrained(config['ref_lm_name']).to(config['device_ref'])
pegasus_tokenizer = PegasusTokenizer.from_pretrained(config['tk_name'])

Some weights of PegasusWithValueHeadModel were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['v_head.bias', 'v_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: unknown error

### W & B Logger

In [ ]:
wandb.init(name='run-5-SARI', project='simp-rl-pegasus', config=config)

In [ ]:
wandb.watch(pegasus_model, log='all')

### Training PPO

In [ ]:
# newsela_tensors = orig_newsela_auto_sents
orig_newsela_auto_sents = orig_newsela_auto_sents[:8]
ref_newsela_auto_sents = ref_newsela_auto_sents[:8]
query_tokens = pegasus_tokenizer(orig_newsela_auto_sents, truncation=True,padding='longest',max_length=config["txt_in_len"], return_tensors="pt")


In [18]:
ppo_trainer = PegasusPPOTrainer(pegasus_model, pegasus_model_ref, **config)
fbs = config['forward_batch_size']

In [19]:
pegasus_model.eval()

PegasusWithValueHeadModel(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(60, 1024)
      (layers): ModuleList(
        (0): PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_aff

In [24]:
for epoch in tqdm(range(int(np.ceil(config["steps"]/config['batch_size'])))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()
    input_dict = dict()
    t0 = time.time()

    #### get a batch from the dataset
    random_batch_indices = random.choices(list(range(0, query_tokens['input_ids'].shape[0])), 
                                          k=config['batch_size'])
    game_data['query'] = [orig_newsela_auto_sents[i] for i in random_batch_indices]
    game_data['references'] = [ref_newsela_auto_sents[i] for i in random_batch_indices]
    input_dict['input_ids'] = query_tokens['input_ids'].index_select(0, torch.tensor(random_batch_indices))
    input_dict['attention_mask'] = query_tokens['attention_mask'].index_select(0, torch.tensor(random_batch_indices))
    query_tensors = transformers.tokenization_utils_base.BatchEncoding(input_dict)
    query_tensors['decoder_input_ids'] = torch.zeros(query_tensors["input_ids"].shape[0], 1, dtype=int)
    query_tensors.to(config['device_ppo'])

#     print("game_data['query'] ", game_data['query'])
#     print("input_dict['input_ids'] ", input_dict['input_ids'])
#     print("input_dict['attention_mask'] ", input_dict['attention_mask'])
#     print("query_tensors ", query_tensors)

    #### get response from pegasus
    t = time.time()
    # total_length = config['txt_in_len']+config['txt_out_len']
    response_tensors_dict = {
        "input_ids": [],
        "attention_mask": [],
        "decoder_input_ids": [],
    }

    for i in range(int(config['batch_size']/fbs)):
        model_input = split_batch_encoding(query_tensors, i*fbs, (i+1)*fbs)
#         print("model_input: ", model_input)
        respond_to_batch_pegasus(pegasus_model, model_input, txt_len=config['txt_out_len'])
        response_tensors_dict["input_ids"].append(model_input['input_ids'])
        response_tensors_dict["attention_mask"].append(model_input['attention_mask'])
        response_tensors_dict["decoder_input_ids"].append(model_input['decoder_input_ids'])

    response_tensors_dict['input_ids'] = torch.cat(response_tensors_dict["input_ids"])
    response_tensors_dict['attention_mask'] = torch.cat(response_tensors_dict["attention_mask"])
    response_tensors_dict['decoder_input_ids'] = torch.cat(response_tensors_dict["decoder_input_ids"])


    game_data['response'] = [pegasus_tokenizer.decode(response_tensors_dict['decoder_input_ids'][i, :]) for i in range(config['batch_size'])]
    timing['time/get_response'] = time.time()-t

#     print("response_tensors_dict ", response_tensors_dict)
    print("game_data['response']:  ", game_data['response'])
    print("game_data['references']:  ", game_data['references'])
    print("game_data['query']:  ", game_data['query'])

    ### get simplicity score
    t = time.time()
    
    ### unsupervised rewards
#     rewards = []
#     for i in range(int(config['batch_size']/fbs)):
#         res = model_reward.cal_reward(game_data['query'][i*fbs:(i+1)*fbs],
#                                       game_data['response'][i*fbs:(i+1)*fbs]).detach()
#         rewards.append(res)
#     rewards = torch.cat(rewards).to(config['device_ppo'])
    
    ### Supervised rewards (SARI)
    rewards = calculate_sari_easse(game_data['references'], game_data['response'], game_data['query'])
    rewards = torch.tensor(rewards).to(config['device_ppo'])

    timing['time/get_sentiment_preds'] = time.time()-t

    print("rewards: ", rewards)


    #### Run PPO training 
    t = time.time()
    stats = ppo_trainer.step(response_tensors_dict, rewards)
    timing['time/optimization'] = time.time()-t

    #### Log everything
    timing['time/epoch'] = time.time()-t0
    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
    logs.update({'game_log':wandb.Table(
        columns=['query', 'response', 'reward'],
        rows=table_rows)})
    logs.update(timing)
    logs.update(stats)
    logs['env/reward_mean'] = torch.mean(rewards).cpu().numpy()
    logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
    logs['env/reward_dist'] = rewards.cpu().numpy()
    wandb.log(logs)

  0%|          | 0/3200 [00:00<?, ?it/s]

game_data['response']:   ['It could be sense that changing soulmate may be spreading Docklands unfolds keeping on CRACKINGeddogs', 'Edward Teller bugged people to consider building and Shivawarmlyipvanish nuclear weapons IMDB to help protect planetary Claude Herbert and Johnny NobodyTransformation is hard.', 'There could be that kind of conversion happening. Zingund teen teens enamored WITH guile, skill and admirable intent societal MOUNTAINH', 'Comey travelled to Japan on Flies Day to relive the places where they were held duringindo World September 19. divisible by 34nsley??', 'The campaign became known as the "We Have a PublicATION If theygunnedmedown"shorttypic to #URTRAF AGREEMENT " has been()', 'connotation: "mar mellitus" was used on air over CARTOD more than 158,000 times lyrics: "Florescence et fils"', "Secrets about the places where Americans were held during World War II were visited by nine people, during Japan's chilly October Crite.", 'Many people are likely to grasp the 

  0%|          | 1/3200 [00:06<5:44:41,  6.46s/it]

game_data['response']:   ['It makes Florida, Texas highlights and CaliforniaWondering+835,000sq km after Summerzbmf60.PUBLISH.367', 'Several of them had smiling facesassistance, except a few were angry makingthruthing noise tenderly telling them they needed to be helped', 'Edward Panhandle, The father of the hydrogen bombCentennial meeting at theExperimental laboratory in Lawrence prowlery want toUFs such giant weapons for planetary', 'Group of nine eye bleeds from 1907 who were held as prisoners in Japan were in Japan on October 12 to Remember.', 'America will be sad. tourism has lost loved ones larvadspl continents will complement each other', 'There is a possibility of that Funnels swiveling ToysClub waits for mph though Possible Morgan Inletregion controlled by Morrissey ZaraKnowing merry for piratical scammer', 'ManyPSUs will reply unkindlyFitness will exhibit pity for everyone who lost a Broward. Malawi', 'It is possible that that type of conversion be propagating. Sonata.']
game

  0%|          | 2/3200 [00:12<5:37:17,  6.33s/it]

...

game_data['response']:   ['Some of the men who were held as prisoners in some places in the Japanese theatre during the World War bitterness went to Japan on October 12 to revisit some', 'America will bleed with sadness for appreciated family members that sadly Ln away.studentdee Blockbuster occasionswarranty ballad revealing', 'The native bear population in the assist has increased greatly in the years since steep protections. delays in the 1978 elections and delays in the 1980s reduce', 'It makes pleasant places like Florida, California, and Texas during the summer.', 'After Edward Teller had shaped how the hydrogen bomb is used, he urged his Gavilane colleagues to consider shaker new nuclear weapons for planetary protection.', 'Those who have lost a loved one will be mourned in America. faultless filiality is necessary prey without guile,', 'America expects everyone to be sad for their loved ones who pass away.t.', 'Conversion is0:40 and may be spreading.forums aggregator']
game_dat

  1%|▏         | 40/3200 [04:36<5:47:55,  6.61s/it]

game_data['response']:   ['Nine American servicemen went back in time to revisit territories they 2014. Woodland pa 9470) was held in during World War II.', 'It makes cumin states like Florida, and Texas great climates in summer.2.45 Partner reported on 16 June Albaniann thinking Gallop in Bostonhydro graph', 'Everyone will be sad for those that have passed. branching out, calling them replaceable Leapfrogs held for providing adaptive workforce technologies, skills and', 'The population of greasier bears in the convolutions has been increased since protections were imposed.alingam', 'Everyone who has friends who has loved ones who have lost will be sad in America. soothed wholesome and apple deprived Realty', 'The protections on the bears were imposed in 1975, and the population has increased substantially since. provided', 'The phrase has been used more than 158,000 times since it was used as a slogan for the social media campaign. Passover was over', 'The population of the grizzle Vi

  1%|▏         | 41/3200 [04:46<6:08:02,  6.99s/it]


KeyboardInterrupt: 

In [48]:
game_data['query'], game_data['response']

(["The Freer rotates its art ; every six months the works are swapped out for new pieces and the originals wo n't appear again for five years .",
  "The Freer rotates its art ; every six months the works are swapped out for new pieces and the originals wo n't appear again for five years ."],
 ['Every six months, the paintings are replaced with something new and the originals are back again. 1869.Hate to say, but',
  'The artwork isCllr tax permonth rotation, when works draught are swapped and originals are reinstalled, the syllabic art stays at LOre per monthly'])

In [26]:
from easse.sari import corpus_sari, get_corpus_sari_operation_scores

def calculate_sari_easse(ref_sents, sys_sents, orig_sents):

    sari_scores = []

    for (ref, sys, orig) in zip(ref_sents, sys_sents, orig_sents):
        print([ref], [sys])
        
        add, keep, delete = get_corpus_sari_operation_scores(orig_sents=[orig], sys_sents=[sys],
                                                             refs_sents=[[ref]])
        overal_sari = (add + keep + delete) / 3
        sari_scores.append(overal_sari)

#         print(f'overal sari:{overal_sari}\
#         add: {add}, keep: {keep}, delete: {delete}')

    return sari_scores